# UDS Data Dictionary Exploration and Analysis

This notebook investigates 472 NACC cases with neuropathological slide data

### Task the first

Investigate neuropsych testing results 

In [ ]:
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../../data-files/investigator_nacc67.csv')
df.head()

# TODO: Extract Npsy columns for inspection
# TODO: Discover tests given
# TODO: Seek inter-test correlations

In [ ]:
column_names = df.columns.tolist()
print(f"\nNumber of columns: {len(column_names)}")

In [ ]:
import pdfplumber
import pandas as pd

# Open the PDF and inspect the table structure
with pdfplumber.open("../../data-files/rdd_uds.pdf") as pdf:
    print(f"Total pages: {len(pdf.pages)}\n")
    
    # Check first few pages for tables
    for page_num in range(23, 28):
        page = pdf.pages[page_num]
        tables = page.extract_tables()
        
        if tables:
            print(f"Found {len(tables)} table(s) on page {page_num + 1}")
            print(f"First few rows:")
            for i, row in enumerate(tables[0][:3]):
                print(f"  Row {i}: {row}")
            print("\n")

In [ ]:
# Extract all tables from the PDF
all_data = []

with pdfplumber.open("../../data-files/rdd_uds.pdf") as pdf:
    for page_num in range(23,28):
        page = pdf.pages[page_num]
        tables = page.extract_tables()
        
        for table in tables:
            if table:
                for row in table:
                    # Skip empty rows or header rows
                    if len(row) < 3:
                        continue
                    if row[2] == 'Variable name':
                        continue
                    
                    # Add the row if variable name exists
                    if row[2] and row[2].strip():
                        all_data.append(row)

print(f"Total rows extracted: {len(all_data)}")
print(f"\nFirst few rows:")
for i in range(min(5, len(all_data))):
    print(all_data[i])

In [ ]:
print(len(all_data))

In [ ]:
# Filter for C1 and C2 Neuropsychological Battery
c1_c2_data = []

for row in all_data:
    # The form field can be in column 0 or 1
    form_field = ""
    if row[0]:
        form_field = row[0]
    elif len(row) > 1 and row[1]:
        form_field = row[1]
    
    # Check if it mentions C1 or C2 with Neuropsychological Battery
    if form_field:
        if ('C1' in form_field or 'C2' in form_field):
            c1_c2_data.append(row)

print(f"Rows matching C1 or C2 Neuropsychological Battery: {len(c1_c2_data)}")

In [ ]:
# Convert to DataFrame
df_filtered = pd.DataFrame(c1_c2_data, 
                          columns=['Form1', 'Form2', 'Variable name', 
                                  'Short descriptor', 'Variable type', 'Source'])

# Display first few rows
print(df_filtered.head(10))

# Check shape
print(f"\nShape: {df_filtered.shape}")

# Check for any issues
print(f"\nNull values:\n{df_filtered.isnull().sum()}")

In [ ]:
df_filtered

In [ ]:
# Extract just the variable names
variable_names = df_filtered['Variable name'].tolist()

print(f"Total variables: {len(variable_names)}")
print("\nVariable names:")
for var in variable_names:
    print(f"  - {var}")

In [ ]:
df_main = pd.read_csv('../../data-files/investigator_nacc67.csv')

available_vars = [var for var in variable_names if var.lower() in [col.lower() for col in df_main.columns]]
missing_vars = [var for var in variable_names if var.lower() not in [col.lower() for col in df_main.columns]]


print(f'Variables available in dataset: {len(available_vars)}')
print(f'Variables missing from dataset: {len(missing_vars)}')

if missing_vars:
    print('\nMissing variables:')
    for var in missing_vars:
        print(f'  - {var}')
        
        
cols_to_keep = [col for col in df_main.columns if any(col.lower() == var.lower() for var in variable_names)]
df_filtered_main = df_main[cols_to_keep]

print(f"\nFiltered dataset shape: {df_filtered_main.shape}")

In [ ]:
import pandas as pd
from itertools import zip_longest

# For lists of different lengths, pad with None
dff = pd.DataFrame({
    'available_vars': list(available_vars),
    'cols_to_keep': list(cols_to_keep) + [None] * (len(available_vars) - len(cols_to_keep))
})

dff

In [ ]:
dropped = [var for var in ]

In [ ]:
# Check for duplicates in variable_names (the list from your PDF)
from collections import Counter

# Count occurrences of each variable (case-insensitive)
var_counts = Counter([var.lower() for var in variable_names])

# Find duplicates
duplicates = {var: count for var, count in var_counts.items() if count > 1}

print(f"Duplicate variables in your PDF extraction: {len(duplicates)}")
if duplicates:
    print("\nDuplicate variables:")
    for var, count in duplicates.items():
        print(f"  {var}: appears {count} times")
        # Show the actual different versions
        versions = [v for v in variable_names if v.lower() == var]
        print(f"    Versions: {versions}")

In [ ]:
# Check the values in NACCMMSE and NACCMOCA
print("NACCMMSE value counts:")
print(df_filtered_main['NACCMMSE'].value_counts().sort_index())
print(f"\nNACCMOCA value counts:")
print(df_filtered_main['NACCMOCA'].value_counts().sort_index())

# Create version indicators
# UDS v2: Has MMSE data (not -4) and no MoCA data (-4)
# UDS v3: Has MoCA data (not -4) and possibly MMSE data too

df_filtered_main['has_MMSE'] = df_filtered_main['NACCMMSE'] != -4
df_filtered_main['has_MOCA'] = df_filtered_main['NACCMOCA'] != -4

# Split into groups
v2_only = df_filtered_main[df_filtered_main['has_MMSE'] & ~df_filtered_main['has_MOCA']]
v3_only = df_filtered_main[~df_filtered_main['has_MMSE'] & df_filtered_main['has_MOCA']]
both = df_filtered_main[df_filtered_main['has_MMSE'] & df_filtered_main['has_MOCA']]
neither = df_filtered_main[~df_filtered_main['has_MMSE'] & ~df_filtered_main['has_MOCA']]

# Print summary
print("\n" + "="*60)
print("DATA AVAILABILITY SUMMARY")
print("="*60)
print(f"Total participants: {len(df_filtered_main)}")
print(f"\nUDS v2 (MMSE only): {len(v2_only)} ({len(v2_only)/len(df_filtered_main)*100:.1f}%)")
print(f"UDS v3 (MoCA only): {len(v3_only)} ({len(v3_only)/len(df_filtered_main)*100:.1f}%)")
print(f"Both MMSE and MoCA: {len(both)} ({len(both)/len(df_filtered_main)*100:.1f}%)")
print(f"Neither test: {len(neither)} ({neither/len(df_filtered_main)*100:.1f}%)")
print(f"\nVerification: {len(v2_only) + len(v3_only) + len(both) + len(neither)} = {len(df_filtered_main)}")

# Additional analysis
print("\n" + "="*60)
print("DETAILED BREAKDOWN")
print("="*60)
print(f"Participants with ANY MMSE data: {df_filtered_main['has_MMSE'].sum()}")
print(f"Participants with ANY MoCA data: {df_filtered_main['has_MOCA'].sum()}")
print(f"Participants with usable data (v2 or v3): {len(v2_only) + len(v3_only) + len(both)}")
print(f"Participants with NO test data: {len(neither)}")

In [ ]:
# Create v2 dataset (participants with MMSE, exclude MoCA columns if desired)
df_v2 = v2_only.copy()

# Create v3 dataset (participants with MoCA, exclude MMSE columns if desired)
df_v3 = v3_only.copy()

# If you want to include "both" group, decide which version to assign them to
# Option 1: Add "both" to v3 (assuming later version takes precedence)
df_v3_with_both = pd.concat([v3_only, both], ignore_index=True)

print(f"\nv2 dataset shape: {df_v2.shape}")
print(f"v3 dataset shape: {df_v3.shape}")
print(f"v3 dataset (including 'both'): {df_v3_with_both.shape}")

In [ ]:
import matplotlib.pyplot as plt

# Create bar chart
categories = ['v2 Only\n(MMSE)', 'v3 Only\n(MoCA)', 'Both Tests', 'Neither']
counts = [len(v2_only), len(v3_only), len(both), len(neither)]

plt.figure(figsize=(10, 6))
bars = plt.bar(categories, counts, color=['#2ecc71', '#3498db', '#f39c12', '#e74c3c'])
plt.ylabel('Number of Participants')
plt.title('Distribution of UDS Versions by Test Availability')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{count}\n({count/len(df_filtered_main)*100:.1f}%)',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Create a crosstab to see the relationship
crosstab = pd.crosstab(
    df_filtered_main['has_MMSE'], 
    df_filtered_main['has_MOCA'],
    margins=True,
    margins_name='Total'
)
crosstab.index = ['No MMSE', 'Has MMSE', 'Total']
crosstab.columns = ['No MoCA', 'Has MoCA', 'Total']

print("\nCrosstabulation:")
print(crosstab)

In [ ]:
# Create dataframe with just the "neither" group
df_neither = neither.copy()

print(f"Total rows in 'neither' group: {len(df_neither)}")
print(f"Total columns: {df_neither.shape[1]}")

# Check how much data exists in this subset
print("\n" + "="*60)
print("DATA AVAILABILITY IN 'NEITHER' GROUP")
print("="*60)

# Count non-null values for each column
non_null_counts = df_neither.count()
print("\nColumns with ANY data (not null and not -4):")

for col in df_neither.columns:
    # Count values that are not null AND not -4
    valid_data = df_neither[col][(df_neither[col].notna()) & (df_neither[col] != -4)]
    count = len(valid_data)
    percentage = (count / len(df_neither)) * 100
    
    if count > 0:
        print(f"  {col}: {count} rows ({percentage:.1f}%)")

In [ ]:
# Check for completely empty rows (all values are NaN or -4)
def is_row_empty(row):
    # Exclude the helper columns we added (has_MMSE, has_MOCA)
    data_cols = [col for col in row.index if col not in ['has_MMSE', 'has_MOCA']]
    for val in row[data_cols]:
        if pd.notna(val) and val != -4:
            return False
    return True

completely_empty = df_neither.apply(is_row_empty, axis=1)

print(f"\nCompletely empty rows (all -4 or NaN): {completely_empty.sum()}")
print(f"Rows with at least some data: {(~completely_empty).sum()}")

In [ ]:
# Rank columns by data availability
data_summary = []

for col in df_neither.columns:
    if col not in ['has_MMSE', 'has_MOCA']:
        valid_count = len(df_neither[col][(df_neither[col].notna()) & (df_neither[col] != -4)])
        data_summary.append({
            'Column': col,
            'Valid_Data_Count': valid_count,
            'Percentage': (valid_count / len(df_neither)) * 100
        })

df_summary = pd.DataFrame(data_summary).sort_values('Valid_Data_Count', ascending=False)

print("\nTop 20 columns with most data:")
print(df_summary.head(20).to_string(index=False))

print("\nColumns with NO valid data:")
no_data_cols = df_summary[df_summary['Valid_Data_Count'] == 0]['Column'].tolist()
print(f"Count: {len(no_data_cols)}")
print(no_data_cols)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample 100 random rows for visualization (or all if less than 100)
sample_size = min(100, len(df_neither))
df_sample = df_neither.sample(n=sample_size, random_state=42)

# Create binary matrix: 1 if valid data, 0 otherwise
data_cols = [col for col in df_sample.columns if col not in ['has_MMSE', 'has_MOCA']]
binary_data = df_sample[data_cols].apply(lambda x: ((x.notna()) & (x != -4)).astype(int))

plt.figure(figsize=(15, 8))
plt.imshow(binary_data.T, aspect='auto', cmap='RdYlGn', interpolation='nearest')
plt.colorbar(label='Has Data (1) / No Data (0)')
plt.xlabel('Sample Participants')
plt.ylabel('Variables')
plt.title(f'Data Availability Heatmap (Sample of {sample_size} participants from "neither" group)')
plt.tight_layout()
plt.show()

print(f"\nOverall data density in 'neither' group: {binary_data.values.mean()*100:.2f}%")

In [ ]:
# Look at a few sample rows with some data
rows_with_data = df_neither[~completely_empty]

if len(rows_with_data) > 0:
    print(f"\nSample of rows with some data (showing first 5):")
    print(rows_with_data.head())
    
    # Show which columns have data in these sample rows
    print("\nNon-empty values in first sample row:")
    first_row = rows_with_data.iloc[0]
    for col in first_row.index:
        if col not in ['has_MMSE', 'has_MOCA'] and pd.notna(first_row[col]) and first_row[col] != -4:
            print(f"  {col}: {first_row[col]}")

In [ ]:
rows_with_data